<a href="https://colab.research.google.com/github/draaronv/statistical_learning/blob/master/regression_logistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages('googledrive')
install.packages('httpuv') 
library('googledrive') 
library('httpuv')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘promises’, ‘later’




In [ ]:
if (file.exists("/usr/local/lib/python3.7/dist-packages/google/colab/_ipython.py")) {
  install.packages("R.utils")
  library("R.utils")
  library("httr")
  my_check <- function() {return(TRUE)}
  reassignInPackage("is_interactive", pkgName = "httr", my_check) 
  options(rlang_interactive=TRUE)
}

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.1 (2020-08-26 16:20:06 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.24.0 (2020-08-26 16:11:58 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.11.0 (2021-09-26 08:30:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, inherits, isOpen, nullfile, parse,
    warnings




In [ ]:
x <- drive_get("~/statistical_learning/finalData.csv")
drive_download(x)





✔ The input `path` resolved to exactly 1 file.

File downloaded:

• finalData.csv <id: 1W1wazYCqs5Iiz6cAwFUjo9f3a3kaBxWU>

Saved locally as:

• finalData.csv



In [ ]:
rm(list = ls())

In [ ]:
install.packages("Metrics")
install.packages("rsample")
install.packages("rpart")
install.packages("dplyr")
install.packages("rpart.plot")
install.packages("ipred")
install.packages("caret")
install.packages("caTools")
install.packages("broom")
install.packages("parallel")
install.packages("doParallel")
install.packages("FNN")
install.packages("e1071")
library(e1071)
library(parallel)
library(FNN)
library(doParallel)
library(broom)
library(Metrics)
library(rsample)     # data splitting 
library(dplyr)       # data wrangling
library(rpart)       # performing regression trees
library(rpart.plot)  # plotting regression trees
library(ipred)       # bagging
library(caret)       # bagging
library(caTools)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘parallel’ is a base package, and should not be updated”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Inst

In [ ]:
cluster <- makeCluster(detectCores() - 1) # convention to leave 1 core for OS
registerDoParallel(cluster)

# Data Initilization

In [ ]:
data1 = read.csv("finalData.csv")

In [ ]:
data2=head(data1,100000)

In [ ]:
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
colSums(is.na(data2))

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                 83                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 100000

[1] 99917

[1] 99917

'data.frame':	78506 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 10 levels "Clear","Clouds",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ weather_descriptio

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                  0                   0                   0                   0 
       weather_main weather_description 
                  0                   0

In [ ]:
unique(data2$KILLED_INJURED)

[1]  0  3  1  6  2  4  5  8 10 12  7 18  9 11 16 13 20 15 17 19 14 22 28

## Logistic Regression

### GLM

#### Gaussian

In [ ]:
glm.fit1=glm(KILLED_INJURED~., data=training,family=gaussian())
summary(glm.fit1)
imp <- as.data.frame(varImp(glm.fit1))
imp <- data.frame(overall = imp$Overall,
           names   = rownames(imp))
imp[order(imp$overall,decreasing = T),]


Call:
glm(formula = KILLED_INJURED ~ ., family = gaussian(), data = training)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.3219  -0.3975  -0.3360  -0.2101  17.5653  

Coefficients: (9 not defined because of singularities)
                                                  Estimate Std. Error t value
(Intercept)                                     -1.256e+00  4.699e-01  -2.674
CRASH_DATE                                       5.064e-05  7.747e-06   6.536
CRASH_TIME                                       3.866e-04  6.750e-04   0.573
PERSON_AGE                                      -1.194e-05  2.662e-05  -0.449
PERSON_SEXM                                     -6.912e-02  6.834e-03 -10.114
WEEKDAYMonday                                   -2.442e-02  1.197e-02  -2.040
WEEKDAYSaturday                                  1.978e-02  1.195e-02   1.656
WEEKDAYSunday                                    4.690e-02  1.259e-02   3.724
WEEKDAYThursday                                 -

,overall,names
,<dbl>,<chr>
12,13.3592654,temp
4,10.1136013,PERSON_SEXM
10,7.9096138,WEEKDAYWednesday
1,6.5363108,CRASH_DATE
7,3.7239151,WEEKDAYSunday
15,3.5903387,humidity
11,3.3492751,HolidayTrue
32,2.7361378,weather_descriptionproximity thunderstorm
9,2.5874939,WEEKDAYTuesday


In [ ]:
anova(glm.fit1, test ="Chisq")

,Df,Deviance,Resid. Df,Resid. Dev,Pr(>Chi)
,<int>,<dbl>,<int>,<dbl>,<dbl>
NULL,NA,NA,78505,63501.46,NA
CRASH_DATE,1,46.6617839,78504,63454.80,2.517974e-14
CRASH_TIME,1,11.3733335,78503,63443.43,1.682446e-04
PERSON_AGE,1,0.1960597,78502,63443.23,6.213092e-01
PERSON_SEX,1,83.6144861,78501,63359.62,1.949646e-24
WEEKDAY,6,107.2257458,78495,63252.39,2.397585e-26
Holiday,1,10.3118311,78494,63242.08,3.402070e-04
temp,1,161.3596713,78493,63080.72,1.368520e-45
visibility,1,2.0125500,78492,63078.71,1.134868e-01


In [ ]:
glm.pred <- predict(glm.fit1, test,type="response")
glm.pred= ceiling(glm.pred)
u <- union(glm.pred, test$KILLED_INJURED)
t <- table(factor(glm.pred, u), factor(test$KILLED_INJURED, u))
confusionMatrix(t)

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


Confusion Matrix and Statistics

    
         1     2     0     6     5     3     4     8    10    12     7    18
  1   3262   871 16602    36    70   338   193     9     5    11     8     4
  2      0     2     0     0     0     0     0     0     0     0     0     0
  0      0     0     0     0     0     0     0     0     0     0     0     0
  6      0     0     0     0     0     0     0     0     0     0     0     0
  5      0     0     0     0     0     0     0     0     0     0     0     0
  3      0     0     0     0     0     0     0     0     0     0     0     0
  4      0     0     0     0     0     0     0     0     0     0     0     0
  8      0     0     0     0     0     0     0     0     0     0     0     0
  10     0     0     0     0     0     0     0     0     0     0     0     0
  12     0     0     0     0     0     0     0     0     0     0     0     0
  7      0     0     0     0     0     0     0     0     0     0     0     0
  18     0     0     0     0     0    

#### Poisson

In [ ]:
glm.fit2=glm(KILLED_INJURED~., data=training,family=poisson())
summary(glm.fit2)
imp <- as.data.frame(varImp(glm.fit2))
imp <- data.frame(overall = imp$Overall,
           names   = rownames(imp))
imp[order(imp$overall,decreasing = T),]


Call:
glm(formula = KILLED_INJURED ~ ., family = poisson(), data = training)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6063  -0.8857  -0.8136  -0.6729   9.9407  

Coefficients: (9 not defined because of singularities)
                                                  Estimate Std. Error z value
(Intercept)                                     -6.136e+00  9.102e-01  -6.742
CRASH_DATE                                       1.469e-04  1.450e-05  10.133
CRASH_TIME                                       8.468e-04  1.236e-03   0.685
PERSON_AGE                                      -3.923e-05  5.833e-05  -0.673
PERSON_SEXM                                     -1.847e-01  1.230e-02 -15.017
WEEKDAYMonday                                   -7.207e-02  2.215e-02  -3.254
WEEKDAYSaturday                                  4.955e-02  2.148e-02   2.307
WEEKDAYSunday                                    1.098e-01  2.205e-02   4.982
WEEKDAYThursday                                 -7

,overall,names
,<dbl>,<chr>
12,19.9850119,temp
4,15.0170279,PERSON_SEXM
10,12.2853620,WEEKDAYWednesday
1,10.1332153,CRASH_DATE
15,5.1357760,humidity
7,4.9821605,WEEKDAYSunday
11,4.7961775,HolidayTrue
9,3.9547199,WEEKDAYTuesday
8,3.6958530,WEEKDAYThursday


In [ ]:
anova(glm.fit2, test ="Chisq")

,Df,Deviance,Resid. Df,Resid. Dev,Pr(>Chi)
,<int>,<dbl>,<int>,<dbl>,<dbl>
NULL,NA,NA,78505,95915.27,NA
CRASH_DATE,1,130.1386090,78504,95785.13,3.821393e-30
CRASH_TIME,1,31.2840927,78503,95753.85,2.228985e-08
PERSON_AGE,1,0.6218685,78502,95753.23,4.303538e-01
PERSON_SEX,1,225.9619961,78501,95527.26,4.528946e-51
WEEKDAY,6,298.7749635,78495,95228.49,1.497110e-61
Holiday,1,25.9344800,78494,95202.55,3.532037e-07
temp,1,462.7936597,78493,94739.76,1.185623e-102
visibility,1,6.3349845,78492,94733.43,1.183792e-02


In [ ]:
glm.pred <- predict(glm.fit2, test,type="response")
glm.pred= ceiling(glm.pred)
u <- union(glm.pred, test$KILLED_INJURED)
t <- table(factor(glm.pred, u), factor(test$KILLED_INJURED, u))
confusionMatrix(t)

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


Confusion Matrix and Statistics

    
         1     2     0     6     5     3     4     8    10    12     7    18
  1   3262   871 16602    36    70   338   193     9     5    11     8     4
  2      0     2     0     0     0     0     0     0     0     0     0     0
  0      0     0     0     0     0     0     0     0     0     0     0     0
  6      0     0     0     0     0     0     0     0     0     0     0     0
  5      0     0     0     0     0     0     0     0     0     0     0     0
  3      0     0     0     0     0     0     0     0     0     0     0     0
  4      0     0     0     0     0     0     0     0     0     0     0     0
  8      0     0     0     0     0     0     0     0     0     0     0     0
  10     0     0     0     0     0     0     0     0     0     0     0     0
  12     0     0     0     0     0     0     0     0     0     0     0     0
  7      0     0     0     0     0     0     0     0     0     0     0     0
  18     0     0     0     0     0    

### Cross Validation

#### Gaussian

In [ ]:
set.seed(1)
fitControl <- trainControl(method = "cv",number = 5)
glm.fit1 <- train(KILLED_INJURED~., data = training, 
                 method = "glm", 
                 family="gaussian",
                 trControl = fitControl)
glm.fit1

Generalized Linear Model 

79934 samples
   19 predictor

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 63947, 63948, 63947, 63947, 63947 
Resampling results:

  RMSE       Rsquared     MAE      
  0.8946014  0.008059529  0.5549874


#### Poisson

In [ ]:
set.seed(1)
fitControl1 <- trainControl(method = "cv",number = 5)
glm.fit2 <- train(KILLED_INJURED~., data = training, 
                 method = "glm", 
                 family="poisson",
                 trControl = fitControl1)
glm.fit2

Warning message:
“glm.fit: fitted rates numerically 0 occurred”


Generalized Linear Model 

76121 samples
   20 predictor

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 60897, 60897, 60897, 60896, 60897 
Resampling results:

  RMSE       Rsquared    MAE      
  0.8718115  0.05739358  0.5248261


# Different size

In [ ]:
data2=head(data1,200000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
glm.fit2=glm(KILLED_INJURED~., data=training,family=poisson())
summary(glm.fit2)
imp <- as.data.frame(varImp(glm.fit2))
imp <- data.frame(overall = imp$Overall,
           names   = rownames(imp))
imp[order(imp$overall,decreasing = T),]
glm.pred <- predict(glm.fit2, test,type="response")
glm.pred= ceiling(glm.pred)
u <- union(glm.pred, test$KILLED_INJURED)
t <- table(factor(glm.pred, u), factor(test$KILLED_INJURED, u))
confusionMatrix(t)


CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                 83                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 200000

[1] 199917

[1] 199917

'data.frame':	157077 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ weather_descripti


Call:
glm(formula = KILLED_INJURED ~ ., family = poisson(), data = training)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5246  -0.8669  -0.8054  -0.7143  10.1816  

Coefficients: (10 not defined because of singularities)
                                                  Estimate Std. Error z value
(Intercept)                                     -3.960e+00  6.283e-01  -6.303
CRASH_DATE                                       1.430e-04  1.063e-05  13.453
CRASH_TIME                                      -1.668e-03  9.043e-04  -1.844
PERSON_AGE                                       2.368e-05  3.175e-05   0.746
PERSON_SEXM                                     -1.928e-01  8.791e-03 -21.930
WEEKDAYMonday                                   -3.842e-02  1.601e-02  -2.400
WEEKDAYSaturday                                  7.681e-02  1.550e-02   4.955
WEEKDAYSunday                                    1.485e-01  1.582e-02   9.384
WEEKDAYThursday                                  

,overall,names
,<dbl>,<chr>
12,24.1469637,temp
4,21.9296265,PERSON_SEXM
1,13.4533111,CRASH_DATE
7,9.3840068,WEEKDAYSunday
11,7.9754072,HolidayTrue
6,4.9545597,WEEKDAYSaturday
10,4.9484208,WEEKDAYWednesday
15,4.5511053,humidity
34,3.8096950,weather_descriptionscattered clouds


Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


Confusion Matrix and Statistics

    
         1     2     0     6     5     3     4     8    10    12     7    18
  1   6199  1739 33499    65   170   769   322    18    12    11    24     4
  2      0     0     2     0     0     0     0     0     0     0     0     0
  0      0     0     0     0     0     0     0     0     0     0     0     0
  6      0     0     0     0     0     0     0     0     0     0     0     0
  5      0     0     0     0     0     0     0     0     0     0     0     0
  3      0     0     0     0     0     0     0     0     0     0     0     0
  4      0     0     0     0     0     0     0     0     0     0     0     0
  8      0     0     0     0     0     0     0     0     0     0     0     0
  10     0     0     0     0     0     0     0     0     0     0     0     0
  12     0     0     0     0     0     0     0     0     0     0     0     0
  7      0     0     0     0     0     0     0     0     0     0     0     0
  18     0     0     0     0     0    

In [ ]:
data2=head(data1,500000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
glm.fit2=glm(KILLED_INJURED~., data=training,family=poisson())
summary(glm.fit2)
imp <- as.data.frame(varImp(glm.fit2))
imp <- data.frame(overall = imp$Overall,
           names   = rownames(imp))
imp[order(imp$overall,decreasing = T),]
glm.pred <- predict(glm.fit2, test,type="response")
glm.pred= ceiling(glm.pred)
u <- union(glm.pred, test$KILLED_INJURED)
t <- table(factor(glm.pred, u), factor(test$KILLED_INJURED, u))
confusionMatrix(t)


CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  1                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                442                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 500000

[1] 499558

[1] 499557

'data.frame':	392508 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ weather_descripti


Call:
glm(formula = KILLED_INJURED ~ ., family = poisson(), data = training)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1376  -0.8568  -0.7994  -0.7223  10.5001  

Coefficients: (10 not defined because of singularities)
                                                  Estimate Std. Error z value
(Intercept)                                     -1.791e+00  4.077e-01  -4.393
CRASH_DATE                                       1.163e-04  7.133e-06  16.310
CRASH_TIME                                      -1.451e-03  5.761e-04  -2.519
PERSON_AGE                                      -8.362e-05  3.132e-05  -2.669
PERSON_SEXM                                     -1.871e-01  5.610e-03 -33.352
WEEKDAYMonday                                   -2.483e-02  1.035e-02  -2.400
WEEKDAYSaturday                                  1.086e-01  9.734e-03  11.155
WEEKDAYSunday                                    1.905e-01  9.986e-03  19.072
WEEKDAYThursday                                 -

,overall,names
,<dbl>,<chr>
4,33.3516532,PERSON_SEXM
12,32.9465585,temp
7,19.0722438,WEEKDAYSunday
1,16.3102636,CRASH_DATE
11,12.0809522,HolidayTrue
6,11.1548842,WEEKDAYSaturday
10,7.3353902,WEEKDAYWednesday
20,6.5564105,weather_mainHaze
28,5.9872471,weather_descriptionfew clouds


Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


Confusion Matrix and Statistics

    
         1     0     6     2     5     3     4     8    10    12     7    18
  1  15383 83893   156  4477   367  1865   732    51    21    15    51     7
  0      0     0     0     0     0     0     0     0     0     0     0     0
  6      0     0     0     0     0     0     0     0     0     0     0     0
  2      0     0     0     0     0     0     0     0     0     0     0     0
  5      0     0     0     0     0     0     0     0     0     0     0     0
  3      0     0     0     0     0     0     0     0     0     0     0     0
  4      0     0     0     0     0     0     0     0     0     0     0     0
  8      0     0     0     0     0     0     0     0     0     0     0     0
  10     0     0     0     0     0     0     0     0     0     0     0     0
  12     0     0     0     0     0     0     0     0     0     0     0     0
  7      0     0     0     0     0     0     0     0     0     0     0     0
  18     0     0     0     0     0    

In [ ]:
data2=head(data1,1000000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
glm.fit2=glm(KILLED_INJURED~., data=training,family=poisson())
summary(glm.fit2)
imp <- as.data.frame(varImp(glm.fit2))
imp <- data.frame(overall = imp$Overall,
           names   = rownames(imp))
imp[order(imp$overall,decreasing = T),]
glm.pred <- predict(glm.fit2, test,type="response")
glm.pred= ceiling(glm.pred)
u <- union(glm.pred, test$KILLED_INJURED)
t <- table(factor(glm.pred, u), factor(test$KILLED_INJURED, u))
confusionMatrix(t)


CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 22                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               1106                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 1000000

[1] 998894

[1] 998872

'data.frame':	784828 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ weather_descripti


Call:
glm(formula = KILLED_INJURED ~ ., family = poisson(), data = training)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1867  -0.8558  -0.7964  -0.7226  11.4174  

Coefficients: (10 not defined because of singularities)
                                                  Estimate Std. Error z value
(Intercept)                                     -1.349e+00  2.906e-01  -4.640
CRASH_DATE                                       1.191e-04  5.235e-06  22.752
CRASH_TIME                                       1.827e-03  4.112e-04   4.442
PERSON_AGE                                      -1.136e-04  2.348e-05  -4.836
PERSON_SEXM                                     -1.949e-01  3.969e-03 -49.119
WEEKDAYMonday                                   -3.136e-02  7.240e-03  -4.331
WEEKDAYSaturday                                  1.285e-01  6.829e-03  18.818
WEEKDAYSunday                                    2.329e-01  7.139e-03  32.629
WEEKDAYThursday                                 -

,overall,names
,<dbl>,<chr>
4,49.1193335,PERSON_SEXM
12,39.1662474,temp
7,32.6288981,WEEKDAYSunday
1,22.7517593,CRASH_DATE
11,19.4933977,HolidayTrue
6,18.8183085,WEEKDAYSaturday
15,10.1451444,humidity
10,7.6827204,WEEKDAYWednesday
21,6.7344925,weather_mainMist


Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


Confusion Matrix and Statistics

    
          1      0      6      2      5      3      4      8     10     12
  1   31003 167642    349   8805    766   3615   1515     91     33     26
  0       0      0      0      0      0      0      0      0      0      0
  6       0      0      0      0      0      0      0      0      0      0
  2       0      0      0      0      0      0      0      0      0      0
  5       0      0      0      0      0      0      0      0      0      0
  3       0      0      0      0      0      0      0      0      0      0
  4       0      0      0      0      0      0      0      0      0      0
  8       0      0      0      0      0      0      0      0      0      0
  10      0      0      0      0      0      0      0      0      0      0
  12      0      0      0      0      0      0      0      0      0      0
  7       0      0      0      0      0      0      0      0      0      0
  18      0      0      0      0      0      0      0      0  

In [ ]:
data2=head(data1,2000000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
glm.fit2=glm(KILLED_INJURED~., data=training,family=poisson())
summary(glm.fit2)
imp <- as.data.frame(varImp(glm.fit2))
imp <- data.frame(overall = imp$Overall,
           names   = rownames(imp))
imp[order(imp$overall,decreasing = T),]
glm.pred <- predict(glm.fit2, test,type="response")
glm.pred= ceiling(glm.pred)
u <- union(glm.pred, test$KILLED_INJURED)
t <- table(factor(glm.pred, u), factor(test$KILLED_INJURED, u))
confusionMatrix(t)


CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 36                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               2292                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 2000000

[1] 1997708

[1] 1997672

'data.frame':	1569599 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ weather_descript


Call:
glm(formula = KILLED_INJURED ~ ., family = poisson(), data = training)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.3035  -0.8699  -0.8066  -0.7263  13.9802  

Coefficients: (10 not defined because of singularities)
                                                  Estimate Std. Error z value
(Intercept)                                     -2.618e+00  2.014e-01 -13.003
CRASH_DATE                                       1.754e-04  3.498e-06  50.146
CRASH_TIME                                      -1.167e-03  2.514e-04  -4.644
PERSON_AGE                                      -9.998e-05  1.546e-05  -6.467
PERSON_SEXM                                     -1.909e-01  2.773e-03 -68.842
WEEKDAYMonday                                    5.035e-03  5.064e-03   0.994
WEEKDAYSaturday                                  1.362e-01  4.817e-03  28.267
WEEKDAYSunday                                    2.307e-01  4.866e-03  47.412
WEEKDAYThursday                                 -

,overall,names
,<dbl>,<chr>
4,68.8421084,PERSON_SEXM
12,60.2356372,temp
1,50.1457464,CRASH_DATE
7,47.4121957,WEEKDAYSunday
6,28.2670397,WEEKDAYSaturday
11,22.9932442,HolidayTrue
15,20.6535003,humidity
10,8.7984192,WEEKDAYWednesday
14,7.0123810,pressure


Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


Confusion Matrix and Statistics

    
          1      0      6      2      5      3      4      8     10     12
  1   64070 331946    673  18639   1537   7341   3190    129     73     32
  0       0      0      0      0      0      0      0      0      0      0
  6       0      0      0      0      0      0      0      0      0      0
  2       0      0      0      0      0      0      0      0      0      0
  5       0      0      0      0      0      0      0      0      0      0
  3       0      0      0      0      0      0      0      0      0      0
  4       0      0      0      0      0      0      0      0      0      0
  8       0      0      0      0      0      0      0      0      0      0
  10      0      0      0      0      0      0      0      0      0      0
  12      0      0      0      0      0      0      0      0      0      0
  7       0      0      0      0      0      0      0      0      0      0
  18      0      0      0      0      0      0      0      0  

In [ ]:
data2=data1
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
glm.fit2=glm(KILLED_INJURED~., data=training,family=poisson())
summary(glm.fit2)
imp <- as.data.frame(varImp(glm.fit2))
imp <- data.frame(overall = imp$Overall,
           names   = rownames(imp))
imp[order(imp$overall,decreasing = T),]
glm.pred <- predict(glm.fit2, test,type="response")
glm.pred= ceiling(glm.pred)
u <- union(glm.pred, test$KILLED_INJURED)
t <- table(factor(glm.pred, u), factor(test$KILLED_INJURED, u))
confusionMatrix(t)


CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 39                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               2737                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 2529009

[1] 2526272

[1] 2526233

'data.frame':	1984898 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ weather_descript


Call:
glm(formula = KILLED_INJURED ~ ., family = poisson(), data = training)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.4329  -0.9161  -0.8149   0.3145  14.0159  

Coefficients: (10 not defined because of singularities)
                                                  Estimate Std. Error z value
(Intercept)                                     -7.174e+00  1.650e-01 -43.484
CRASH_DATE                                       4.029e-04  1.957e-06 205.920
CRASH_TIME                                       9.961e-04  2.045e-04   4.871
PERSON_AGE                                      -1.301e-04  1.547e-05  -8.410
PERSON_SEXM                                     -1.737e-01  2.330e-03 -74.547
WEEKDAYMonday                                    1.325e-02  4.226e-03   3.135
WEEKDAYSaturday                                  1.447e-01  4.038e-03  35.841
WEEKDAYSunday                                    2.237e-01  4.089e-03  54.715
WEEKDAYThursday                                 -

,overall,names
,<dbl>,<chr>
1,205.9199451,CRASH_DATE
12,78.0517338,temp
4,74.5466044,PERSON_SEXM
7,54.7149102,WEEKDAYSunday
6,35.8413720,WEEKDAYSaturday
11,25.4482639,HolidayTrue
15,23.0983593,humidity
3,8.4102817,PERSON_AGE
18,6.8388720,weather_mainDrizzle


Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


Confusion Matrix and Statistics

    
          1      2      0      6      5      3      4      8     10     12
  1   92553  26860 403015    893   2021  10575   4483    176    102     32
  2       5      1      2      0      0      0      0      0      0      0
  0       0      0      0      0      0      0      0      0      0      0
  6       0      0      0      0      0      0      0      0      0      0
  5       0      0      0      0      0      0      0      0      0      0
  3       0      0      0      0      0      0      0      0      0      0
  4       0      0      0      0      0      0      0      0      0      0
  8       0      0      0      0      0      0      0      0      0      0
  10      0      0      0      0      0      0      0      0      0      0
  12      0      0      0      0      0      0      0      0      0      0
  7       0      0      0      0      0      0      0      0      0      0
  18      0      0      0      0      0      0      0      0  

# Downsample

In [ ]:
data2=head(data1,100000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
glm.fit2=glm(KILLED_INJURED~., data=training,family=poisson())
summary(glm.fit2)
imp <- as.data.frame(varImp(glm.fit2))
imp <- data.frame(overall = imp$Overall,
           names   = rownames(imp))
imp[order(imp$overall,decreasing = T),]
glm.pred <- predict(glm.fit2, test,type="response")
glm.pred= ceiling(glm.pred)
u <- union(glm.pred, test$KILLED_INJURED)
t <- table(factor(glm.pred, u), factor(test$KILLED_INJURED, u))
confusionMatrix(t)


CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                 83                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 100000

[1] 99917

[1] 99917

[1] 37791

...

'data.frame':	29693 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 33 42 39 38 35 39 35 35 35 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : num  0 0 0 3 3 3 3 3 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 10 levels "Clear","Clouds",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ weather_descriptio


Call:
glm(formula = KILLED_INJURED ~ ., family = poisson(), data = training)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7994  -1.3369  -0.0576   0.2298   8.1354  

Coefficients: (9 not defined because of singularities)
                                                  Estimate Std. Error z value
(Intercept)                                     -1.285e+00  9.102e-01  -1.412
CRASH_DATE                                       5.738e-05  1.434e-05   4.000
CRASH_TIME                                       2.311e-04  1.210e-03   0.191
PERSON_AGE                                      -7.900e-05  5.811e-05  -1.360
PERSON_SEXM                                     -1.419e-01  1.229e-02 -11.551
WEEKDAYMonday                                   -3.876e-02  2.224e-02  -1.743
WEEKDAYSaturday                                  8.395e-02  2.125e-02   3.951
WEEKDAYSunday                                    1.356e-01  2.209e-02   6.136
WEEKDAYThursday                                 -4

,overall,names
,<dbl>,<chr>
12,14.5223491,temp
4,11.5507859,PERSON_SEXM
10,9.2725142,WEEKDAYWednesday
7,6.1360854,WEEKDAYSunday
1,4.0004729,CRASH_DATE
6,3.9511419,WEEKDAYSaturday
15,2.9120085,humidity
11,2.5268870,HolidayTrue
8,2.1061149,WEEKDAYThursday


Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


Confusion Matrix and Statistics

    
        1    2    3    0    6    4    5   10   12    7    8   18
  1  2017  541  184 2129    6   84   40    2    6    2    3    0
  2  1238  367  157 1152   29   91   28    1    8    7    1    4
  3     0    1    0    0    0    0    0    0    0    0    0    0
  0     0    0    0    0    0    0    0    0    0    0    0    0
  6     0    0    0    0    0    0    0    0    0    0    0    0
  4     0    0    0    0    0    0    0    0    0    0    0    0
  5     0    0    0    0    0    0    0    0    0    0    0    0
  10    0    0    0    0    0    0    0    0    0    0    0    0
  12    0    0    0    0    0    0    0    0    0    0    0    0
  7     0    0    0    0    0    0    0    0    0    0    0    0
  8     0    0    0    0    0    0    0    0    0    0    0    0
  18    0    0    0    0    0    0    0    0    0    0    0    0

Overall Statistics
                                          
               Accuracy : 0.2944          
           

In [ ]:
data2=head(data1,500000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
glm.fit2=glm(KILLED_INJURED~., data=training,family=poisson())
summary(glm.fit2)
imp <- as.data.frame(varImp(glm.fit2))
imp <- data.frame(overall = imp$Overall,
           names   = rownames(imp))
imp[order(imp$overall,decreasing = T),]
glm.pred <- predict(glm.fit2, test,type="response")
glm.pred= ceiling(glm.pred)
u <- union(glm.pred, test$KILLED_INJURED)
t <- table(factor(glm.pred, u), factor(test$KILLED_INJURED, u))
confusionMatrix(t)


CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  1                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                442                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 500000

[1] 499558

[1] 499558

[1] 180967

...

'data.frame':	142188 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 33 42 39 38 35 39 35 35 35 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : num  0 0 0 3 3 3 3 3 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ weather_descripti


Call:
glm(formula = KILLED_INJURED ~ ., family = poisson(), data = training)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7171  -1.3467  -0.0327   0.1769   8.5824  

Coefficients: (10 not defined because of singularities)
                                                  Estimate Std. Error z value
(Intercept)                                     -7.307e-02  4.071e-01  -0.179
CRASH_DATE                                       5.004e-05  7.065e-06   7.083
CRASH_TIME                                      -1.317e-03  5.606e-04  -2.349
PERSON_AGE                                      -1.276e-04  3.354e-05  -3.805
PERSON_SEXM                                     -1.459e-01  5.596e-03 -26.070
WEEKDAYMonday                                   -2.537e-02  1.035e-02  -2.452
WEEKDAYSaturday                                  6.853e-02  9.723e-03   7.048
WEEKDAYSunday                                    1.393e-01  9.997e-03  13.932
WEEKDAYThursday                                 -

,overall,names
,<dbl>,<chr>
4,26.07036014,PERSON_SEXM
12,21.29743384,temp
7,13.93174131,WEEKDAYSunday
1,7.08280202,CRASH_DATE
6,7.04837786,WEEKDAYSaturday
11,6.11399634,HolidayTrue
10,5.58984284,WEEKDAYWednesday
20,5.42170636,weather_mainHaze
28,3.97186244,weather_descriptionfew clouds


Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


Confusion Matrix and Statistics

    
         1     2     0     3     4     8     6     5    10    12     7    18
  1  10257  2693 10822  1109   368    26    78   189    10     8    24     2
  2   5216  1762  4819   723   346    24    85   141     6     5    27     6
  0      0     0     0     0     0     0     0     0     0     0     0     0
  3      0     0     0     0     0     0     0     0     0     0     0     0
  4      0     0     0     0     0     0     0     0     0     0     0     0
  8      0     0     0     0     0     0     0     0     0     0     0     0
  6      0     0     0     0     0     0     0     0     0     0     0     0
  5      0     0     0     0     0     0     0     0     0     0     0     0
  10     0     0     0     0     0     0     0     0     0     0     0     0
  12     0     0     0     0     0     0     0     0     0     0     0     0
  7      0     0     0     0     0     0     0     0     0     0     0     0
  18     0     0     0     0     0    

In [ ]:
data2=head(data1,1000000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
glm.fit2=glm(KILLED_INJURED~., data=training,family=poisson())
summary(glm.fit2)
imp <- as.data.frame(varImp(glm.fit2))
imp <- data.frame(overall = imp$Overall,
           names   = rownames(imp))
imp[order(imp$overall,decreasing = T),]
glm.pred <- predict(glm.fit2, test,type="response")
glm.pred= ceiling(glm.pred)
u <- union(glm.pred, test$KILLED_INJURED)
t <- table(factor(glm.pred, u), factor(test$KILLED_INJURED, u))
confusionMatrix(t)


CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 22                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               1106                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 1000000

[1] 998894

[1] 998894

[1] 361994

...

'data.frame':	284407 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  42 42 39 27 39 35 35 35 25 43 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 1 1 ...
 $ KILLED_INJURED     : num  0 0 3 3 3 3 1 1 1 0 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ weather_descripti


Call:
glm(formula = KILLED_INJURED ~ ., family = poisson(), data = training)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7221  -1.3406  -0.0294   0.1775   9.3918  

Coefficients: (10 not defined because of singularities)
                                                  Estimate Std. Error z value
(Intercept)                                      4.951e-01  2.911e-01   1.701
CRASH_DATE                                       5.013e-05  5.190e-06   9.659
CRASH_TIME                                       3.170e-04  3.986e-04   0.795
PERSON_AGE                                      -8.189e-05  2.058e-05  -3.980
PERSON_SEXM                                     -1.457e-01  3.967e-03 -36.732
WEEKDAYMonday                                   -3.321e-02  7.251e-03  -4.581
WEEKDAYSaturday                                  8.664e-02  6.825e-03  12.695
WEEKDAYSunday                                    1.671e-01  7.134e-03  23.418
WEEKDAYThursday                                 -

,overall,names
,<dbl>,<chr>
4,36.732494522,PERSON_SEXM
12,25.675156184,temp
7,23.418129280,WEEKDAYSunday
11,13.104175634,HolidayTrue
6,12.694527104,WEEKDAYSaturday
1,9.659002238,CRASH_DATE
10,7.741819195,WEEKDAYWednesday
15,7.735784375,humidity
28,6.015061449,weather_descriptionfew clouds


Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


Confusion Matrix and Statistics

    
         2     1     3     6     0     4     8     5    10    12     7    18
  2   3419 10055  1451   156  9475   671    53   321    18    13    69     2
  1   5496 21134  2188   182 21490   813    43   357    14     8    64     4
  3      0     0     0     0     0     0     0     0     0     0     0     0
  6      0     0     0     0     0     0     0     0     0     0     0     0
  0      0     0     0     0     0     0     0     0     0     0     0     0
  4      0     0     0     0     0     0     0     0     0     0     0     0
  8      0     0     0     0     0     0     0     0     0     0     0     0
  5      0     0     0     0     0     0     0     0     0     0     0     0
  10     0     0     0     0     0     0     0     0     0     0     0     0
  12     0     0     0     0     0     0     0     0     0     0     0     0
  7      0     0     0     0     0     0     0     0     0     0     0     0
  18     0     0     0     0     0    

In [ ]:
data2=head(data1,2000000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
glm.fit2=glm(KILLED_INJURED~., data=training,family=poisson())
summary(glm.fit2)
imp <- as.data.frame(varImp(glm.fit2))
imp <- data.frame(overall = imp$Overall,
           names   = rownames(imp))
imp[order(imp$overall,decreasing = T),]
glm.pred <- predict(glm.fit2, test,type="response")
glm.pred= ceiling(glm.pred)
u <- union(glm.pred, test$KILLED_INJURED)
t <- table(factor(glm.pred, u), factor(test$KILLED_INJURED, u))
confusionMatrix(t)


CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 36                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               2292                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 2000000

[1] 1997708

[1] 1997708

[1] 746445

...

'data.frame':	586464 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 51 37 27 38 35 39 35 35 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : num  0 0 0 0 3 3 3 3 3 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ weather_descripti


Call:
glm(formula = KILLED_INJURED ~ ., family = poisson(), data = training)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8653  -1.3421  -0.0341   0.1781  10.0410  

Coefficients: (10 not defined because of singularities)
                                                  Estimate Std. Error z value
(Intercept)                                     -2.303e-01  2.011e-01  -1.145
CRASH_DATE                                       7.936e-05  3.441e-06  23.065
CRASH_TIME                                      -1.179e-03  2.445e-04  -4.823
PERSON_AGE                                      -6.442e-05  1.241e-05  -5.193
PERSON_SEXM                                     -1.463e-01  2.770e-03 -52.830
WEEKDAYMonday                                    4.828e-03  5.055e-03   0.955
WEEKDAYSaturday                                  9.820e-02  4.814e-03  20.398
WEEKDAYSunday                                    1.605e-01  4.870e-03  32.953
WEEKDAYThursday                                 -

,overall,names
,<dbl>,<chr>
4,52.8300390,PERSON_SEXM
12,36.8189968,temp
7,32.9529090,WEEKDAYSunday
1,23.0652574,CRASH_DATE
6,20.3976996,WEEKDAYSaturday
11,14.7718518,HolidayTrue
15,13.8610762,humidity
10,7.7026335,WEEKDAYWednesday
28,5.9996242,weather_descriptionfew clouds


Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


Confusion Matrix and Statistics

    
         1     2     0     3     4     5     6    10    12     7     8    18
  1  42302 11363 44273  4256  1650   800   316    35    15   116    59     3
  2  21691  7336 19769  3072  1418   686   312    31    18   159    74     6
  0      0     0     0     0     0     0     0     0     0     0     0     0
  3      0     0     0     0     0     0     0     0     0     0     0     0
  4      0     0     0     0     0     0     0     0     0     0     0     0
  5      0     0     0     0     0     0     0     0     0     0     0     0
  6      0     0     0     0     0     0     0     0     0     0     0     0
  10     0     0     0     0     0     0     0     0     0     0     0     0
  12     0     0     0     0     0     0     0     0     0     0     0     0
  7      0     0     0     0     0     0     0     0     0     0     0     0
  8      0     0     0     0     0     0     0     0     0     0     0     0
  18     0     0     0     0     0    

In [ ]:
data2=data1
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
glm.fit2=glm(KILLED_INJURED~., data=training,family=poisson())
summary(glm.fit2)
imp <- as.data.frame(varImp(glm.fit2))
imp <- data.frame(overall = imp$Overall,
           names   = rownames(imp))
imp[order(imp$overall,decreasing = T),]
glm.pred <- predict(glm.fit2, test,type="response")
glm.pred= ceiling(glm.pred)
u <- union(glm.pred, test$KILLED_INJURED)
t <- table(factor(glm.pred, u), factor(test$KILLED_INJURED, u))
confusionMatrix(t)


CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 39                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               2737                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 2529009

[1] 2526272

[1] 2526272

[1] 1074660

...

'data.frame':	844344 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  82 42 51 37 35 27 38 35 39 35 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 1 2 2 2 2 2 ...
 $ KILLED_INJURED     : num  0 0 0 0 3 3 3 3 3 3 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ weather_descripti


Call:
glm(formula = KILLED_INJURED ~ ., family = poisson(), data = training)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8376  -1.3126  -0.0648   0.2268  11.9969  

Coefficients: (10 not defined because of singularities)
                                                  Estimate Std. Error z value
(Intercept)                                     -2.938e+00  1.649e-01 -17.818
CRASH_DATE                                       1.914e-04  1.908e-06 100.305
CRASH_TIME                                       2.915e-04  2.003e-04   1.455
PERSON_AGE                                      -7.275e-05  1.162e-05  -6.259
PERSON_SEXM                                     -1.355e-01  2.331e-03 -58.135
WEEKDAYMonday                                    1.328e-03  4.233e-03   0.314
WEEKDAYSaturday                                  1.148e-01  4.038e-03  28.427
WEEKDAYSunday                                    1.677e-01  4.091e-03  40.991
WEEKDAYThursday                                 -

,overall,names
,<dbl>,<chr>
1,100.3054427,CRASH_DATE
4,58.1347871,PERSON_SEXM
12,48.1669702,temp
7,40.9905927,WEEKDAYSunday
6,28.4268056,WEEKDAYSaturday
11,17.9295256,HolidayTrue
15,16.4531989,humidity
3,6.2591098,PERSON_AGE
10,5.6861176,WEEKDAYWednesday


Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”


Confusion Matrix and Statistics

    
         1     2     0     3     4     5     8     6    10    12    18     7
  1  57438 15918 67772  6299  2451  1109   102   467    43    23    11   205
  2  34735 11062 24218  4593  2038   827    97   364    54    12     4   210
  0      0     0     0     0     0     0     0     0     0     0     0     0
  3      0     0     0     0     0     0     0     0     0     0     0     0
  4      0     0     0     0     0     0     0     0     0     0     0     0
  5      0     0     0     0     0     0     0     0     0     0     0     0
  8      0     0     0     0     0     0     0     0     0     0     0     0
  6      0     0     0     0     0     0     0     0     0     0     0     0
  10     0     0     0     0     0     0     0     0     0     0     0     0
  12     0     0     0     0     0     0     0     0     0     0     0     0
  18     0     0     0     0     0     0     0     0     0     0     0     0
  7      0     0     0     0     0    